In [2]:
#Import the sampling and analysis modules for a Sobol variance-based 
#sensitivity analysis
from SALib.sample import saltelli
from SALib.analyze import sobol
problem = { 
  'num_vars': 6,
  'names': ['random-seed',
            'grass-regrowth-time',
            'sheep-gain-from-food',
            'wolf-gain-from-food',
            'sheep-reproduce',
            'wolf-reproduce'], 
  'bounds': [[1, 100000],
             [20., 40.], 
             [2., 8.], 
             [16., 32.],
             [2., 8.],
             [2., 8.]]
}
n = 1000
param_values = saltelli.sample(problem, n, calc_second_order=True)

In [3]:
import nl4py
nl4py.startServer()
import time
def simulate(workspace_,experiment):
    workspace_.command("stop")
     #Set the input parameters
    for i, name in enumerate(problem['names']):
        if name == 'random-seed':
            #The NetLogo random seed requires a different syntax
            workspace_.command('random-seed {}'.format(experiment[i]))
        else:
            #Otherwise, assume the input parameters are global variables
            workspace_.command('set {0} {1}'.format(name, experiment[i]))
    workspace_.command('setup')
    workspace_.scheduleReportersAndRun(["ticks",'count sheep','count wolves'], 0,1,100,"go")    

Dependencies installed successfully! 
Start the NetLogoControllerServer using nl4py.startServer()
Shutting down old server instance...
Server started.


In [6]:
def measureExecutionTime(experiment):
    startTime = int(round(time.time() * 1000))
    nl4py.deleteAllHeadlessWorkspaces() 
    runsDone = 0
    runsStarted = 0
    runsNeeded = experiment.shape[0]
    allResults = []
    for i in range(0,8):
        workspace = nl4py.newNetLogoHeadlessWorkspace()
        workspace.openModel('Wolf Sheep Predation.nlogo')
        simulate(workspace,experiment[runsStarted])
        runsStarted = runsStarted + 1
    while (runsDone < runsNeeded):
        for workspace in nl4py.getAllHeadlessWorkspaces():
            newResults = workspace.getScheduledReporterResults()
            if len(newResults) > 0:
                allResults.extend(newResults)
                runsDone = runsDone + 1
                if runsStarted < runsNeeded:
                    simulate(workspace,experiment[runsStarted])
                    runsStarted = runsStarted + 1
    stopTime = int(round(time.time() * 1000))
    print(stopTime - startTime)
    for workspace in nl4py.getAllHeadlessWorkspaces():
        workspace.command("stop")
    nl4py.deleteAllHeadlessWorkspaces() 

In [7]:
for j in range (0,10):
    measureExecutionTime(param_values)

385711
390416
396206
398289
400485
403833
406358
415949
425646
427356
